# Demo: Serenity Derivatives API - Yield Curve and Vol Surface

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to fetch yield curves and volatility
surfaces.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
#%run -i init_demo.py

In [ ]:
# to change back to run init_demo.py when ready
config_id = 'athansor'
from serenity_sdk.client import SerenityApiProvider, SerenityClient
from serenity_sdk.config import load_local_config
config = load_local_config(config_id)
client = SerenityClient(config)
api = SerenityApiProvider(client)

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion

# helper modules
from serenity_sdk.renderers.derivatives.widget_tools import YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import YieldCurveTablePlot, VolatilitySurfaceTablePlot

# plot parameters
plt.rcParams['font.size'] = '16'

In [ ]:
# Pick a datetime.
import pytz
end_dt =datetime.utcnow()
start_dt = end_dt - timedelta(days=1)

# Yield Curves

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
versions = api.pricer().get_available_yield_curve_versions()

In [ ]:
versions = api.pricer().get_available_yield_curve_versions(
    start_datetime=start_dt, end_datetime=end_dt)
vtc_yc = YieldCurveVersionTimeChooser(versions)

print('Pick the curve and as-of-time from the drop-down menus')
display(vtc_yc.get_widget_to_display())

## Now, load the selected version

In [ ]:
id_selected, as_of_time_selected = vtc_yc.get_id_as_of_time()
print(f'Loading id={id_selected}, as-of-time={as_of_time_selected}')

# load the version through API
yc_selected = api.pricer().get_yield_curve_version(id_selected, as_of_time_selected)

# use a helper object to show the content
yc_tp = YieldCurveTablePlot(yc_selected)

### Peeking data

In [ ]:
# peeking raw data
yc_tp.raw_pts.head(2)

In [ ]:
# peeking interpolated data
yc_tp.interpolated_curve.head(2)

## Plotting
* Top: The flat-forward interpolation method is used to get the interpolated points.
* Bottom: It is equivalent to the log-linear interpolation in the discount factor.

In [ ]:
yc_tp.plot()
plt.show()

# Volatility Surfaces

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
# Load available vol surfaces
versions = api.pricer().get_available_volatility_surface_versions(
    start_datetime=start_dt, end_datetime=end_dt)
vtc_vol =  VolatilitySurfaceVersionTimeChooser(versions)

print('Pick the vol surface and build time from the drop-down menus')
display(vtc_vol.get_widget_to_display())

## Now, load the selected version

In [ ]:
id_selected, as_of_time_selected = vtc_vol.get_id_as_of_time()
print(f'Loading id={id_selected}, as-of-time={as_of_time_selected}')

# load the version through API
vs_selected = api.pricer().get_volatility_surface_version(id_selected, as_of_time_selected)

# use a helper object to show the content 
vs_tp = VolatilitySurfaceTablePlot(vs_selected)

### Peeking data

In [ ]:
# peeking raw data
vs_tp.raw_pts.head(2)

In [ ]:
# peeking interpolated data
vs_tp.interpolated_surface.head(2)

## Plotting
For each expiries, show a smile curve with input data (dots)

In [ ]:
vs_tp.plot()


# END